In [1]:
import os

In [2]:
%pwd

'c:\\Users\\quamr\\OneDrive\\Desktop\\project\\gemstonePricePrediction\\research'

In [3]:
os.chdir('c:\\Users\\quamr\\OneDrive\\Desktop\\project\\gemstonePricePrediction')

In [4]:
%pwd

'c:\\Users\\quamr\\OneDrive\\Desktop\\project\\gemstonePricePrediction'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
  root_dir: Path
  source_URL: str
  local_data_file: Path
  unzip_dir: Path

In [6]:
from src.gemstonePricePrediction.constants import *
from src.gemstonePricePrediction.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
            self, 
            config_filepath=CONFIG_FILE_PATH, 
            params_filepath=PARAMS_FILE_PATH,
            schema_filepath=SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(config.root_dir),
            source_URL=config.source_URL,
            local_data_file=Path(config.local_data_file),
            unzip_dir=Path(config.unzip_dir)
        )

        return data_ingestion_config

In [8]:
import os
import urllib.request as request
import zipfile
from gemstonePricePrediction.logger import logging
from gemstonePricePrediction.utils.common import get_size


In [9]:
class DataIngestion:
  def __init__(self, config: DataIngestionConfig):
      self.config = config

  def download_file(self,) -> str:
      if not os.path.exists(self.config.local_data_file):
        filename, headers = request.urlretrieve(
            url=self.config.source_URL,
            filename=self.config.local_data_file
        )
        logging.info(
            f"File: {filename} downloaded with following info: \n{headers}"
        )
      else:
        logging.info(
            f"File already exists of size: {get_size(Path(self.config.local_data_file))}"
        )

  def extract_zip_file(self,) -> None:
      """
      zip_file_path: str
      Extracts the zip file to the unzip directory
      Function return type: None
      """
      unzip_path = self.config.unzip_dir
      os.makedirs(unzip_path, exist_ok=True)
      with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
          zip_ref.extractall(unzip_path)


In [10]:
try:
  config = ConfigurationManager()
  data_ingestion_config = config.get_data_ingestion_config()
  data_ingestion = DataIngestion(config=data_ingestion_config)
  data_ingestion.download_file()
  data_ingestion.extract_zip_file()
except Exception as e:
  raise e


[2026-02-10 18:53:00,208: INFO: common: yaml file loaded successfully from: C:\Users\quamr\OneDrive\Desktop\project\gemstonePricePrediction\config\config.yaml]
[2026-02-10 18:53:00,306: INFO: common: yaml file loaded successfully from: C:\Users\quamr\OneDrive\Desktop\project\gemstonePricePrediction\params.yaml]
[2026-02-10 18:53:00,310: INFO: common: yaml file loaded successfully from: C:\Users\quamr\OneDrive\Desktop\project\gemstonePricePrediction\schema.yaml]
[2026-02-10 18:53:00,312: INFO: common: created directory at: artifacts]
[2026-02-10 18:53:00,315: INFO: common: created directory at: artifacts/data_ingestion]
[2026-02-10 18:53:06,937: INFO: 95082607: File: artifacts\data_ingestion\gemstone.zip downloaded with following info: 
Connection: close
Content-Length: 16739931
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "0faceee890ef6b94e489da9f8d4cc5de02f91ddde0239579ea019970a1df2006"
S